# RAG Based on RAG-Fusion
- Hybrid Search
- ReRanker
- RAG-Fusion

## Setting
 - Auto Reload
 - path for utils

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys, os
module_path = "../../.."
sys.path.append(os.path.abspath(module_path))

## 1. Bedrock Client 생성

In [3]:
import json
import boto3
from pprint import pprint
from termcolor import colored
from utils import bedrock, print_ww
from utils.bedrock import bedrock_info

### ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----
- os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
- os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
- os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."
- os.environ["BEDROCK_ENDPOINT_URL"] = "<YOUR_ENDPOINT_URL>"  # E.g. "https://..."

In [44]:
boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

aws_region = os.environ.get("AWS_DEFAULT_REGION", None)
print (colored("\n== FM lists ==", "green"))
pprint (bedrock_info.get_list_fm_models())

Create new client
  Using region: None
  Using profile: None
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-west-2.amazonaws.com)

== FM lists ==
{'Claude-Instant-V1': 'anthropic.claude-instant-v1',
 'Claude-V1': 'anthropic.claude-v1',
 'Claude-V2': 'anthropic.claude-v2',
 'Claude-V2-1': 'anthropic.claude-v2:1',
 'Claude-V3-5-Sonnet': 'anthropic.claude-3-5-sonnet-20240620-v1:0',
 'Claude-V3-Haiku': 'anthropic.claude-3-haiku-20240307-v1:0',
 'Claude-V3-Opus': 'anthropic.claude-3-sonnet-20240229-v1:0',
 'Claude-V3-Sonnet': 'anthropic.claude-3-sonnet-20240229-v1:0',
 'Cohere-Embeddings-En': 'cohere.embed-english-v3',
 'Cohere-Embeddings-Multilingual': 'cohere.embed-multilingual-v3',
 'Command': 'cohere.command-text-v14',
 'Command-Light': 'cohere.command-light-text-v14',
 'Jurassic-2-Mid': 'ai21.j2-mid-v1',
 'Jurassic-2-Ultra': 'ai21.j2-ultra-v1',
 'Llama2-13b-Chat': 'meta.llama2-13b-chat-v1',
 'Titan-Embeddings-G1': 'amazon.titan-embed-text-v1',
 'T

## 2. Titan Embedding 및 LLM 인 Claude-v3.5 모델 로딩

### LLM 로딩 (Claude-v3.5)

In [5]:
from langchain_aws import ChatBedrock
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [6]:
llm_text = ChatBedrock(
    model_id=bedrock_info.get_model_id(model_name="Claude-V3-5-Sonnet"),
    client=boto3_bedrock,
    model_kwargs={
        "max_tokens": 1024,
        "stop_sequences": ["\n\nHuman"],
    },
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)
llm_text

ChatBedrock(callbacks=[<langchain_core.callbacks.streaming_stdout.StreamingStdOutCallbackHandler object at 0x7fba3a6be710>], client=<botocore.client.BedrockRuntime object at 0x7fba4a4a65f0>, model_id='anthropic.claude-3-5-sonnet-20240620-v1:0', model_kwargs={'max_tokens': 1024, 'stop_sequences': ['\n\nHuman']}, streaming=True)

### Embedding 모델 선택

In [45]:
from langchain_aws import BedrockEmbeddings

In [46]:
llm_emb = BedrockEmbeddings(
    client=boto3_bedrock,
    model_id=bedrock_info.get_model_id(model_name="Titan-Text-Embeddings-V2")
)
dimension = 1024 #1536
print("Bedrock Embeddings Model Loaded")

Bedrock Embeddings Model Loaded


## 3. Depoly ReRanker model (if needed)

In [10]:
import json
import sagemaker
from sagemaker.huggingface import HuggingFaceModel

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [11]:
depoly = False

In [12]:
if depoly:

    try:
        role = sagemaker.get_execution_role()
    except ValueError:
        iam = boto3.client('iam')
        role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

    # Hub Model configuration. https://huggingface.co/models
    hub = {
        'HF_MODEL_ID':'BAAI/bge-reranker-large',
        'HF_TASK':'text-classification'
    }

    # create Hugging Face Model Class
    huggingface_model = HuggingFaceModel(
        transformers_version='4.26.0',
        pytorch_version='1.13.1',
        py_version='py39',
        env=hub,
        role=role, 
    )

    # deploy model to SageMaker Inference
    predictor = huggingface_model.deploy(
        initial_instance_count=1, # number of instances
        instance_type='ml.g5.xlarge' # instance type
    )

    print(f'Accept: {predictor.accept}')
    print(f'ContentType: {predictor.content_type}')
    print(f'Endpoint: {predictor.endpoint}')

#### Save reranker endpoint to Parameter Store

In [13]:
import boto3
from utils.ssm import parameter_store

In [14]:
region=boto3.Session().region_name
pm = parameter_store(region)

In [15]:
pm.put_params(
    key="reranker_endpoint",
    value=f'{predictor.endpoint}',
    overwrite=True,
    enc=False
)

NameError: name 'predictor' is not defined

## 4. Invocation (prediction)

In [16]:
runtime_client = boto3.Session().client('sagemaker-runtime')
print (f'runtime_client: {runtime_client}')

runtime_client: <botocore.client.SageMakerRuntime object at 0x7fb9c8e9a890>


In [17]:
endpoint_name = pm.get_params(
    key="reranker_endpoint",
    enc=False
)
deserializer = "application/json"

In [18]:
payload = json.dumps(
    {
        "inputs": [
            {"text": "I hate you", "text_pair": "I don't like you"},
            {"text": "He hates you", "text_pair": "He like you"}
        ]
    }
)

In [19]:
%%time
response = runtime_client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType="application/json",
    Accept=deserializer,
    Body=payload
)
## deserialization
out = json.loads(response['Body'].read().decode()) ## for json
print (f'Response: {out}')

Response: [{'label': 'LABEL_0', 'score': 0.9987371563911438}, {'label': 'LABEL_0', 'score': 0.0028581616934388876}]
CPU times: user 14.3 ms, sys: 0 ns, total: 14.3 ms
Wall time: 86.1 ms


## 5. LangChainmOpenSearch VectorStore 정의
### 선수 조건
- 01_preprocess_docs/02_load_docs_opensearch.ipynb를 통해서 OpenSearch Index 가 생성이 되어 있어야 합니다.
#### [중요] 아래에 aws parameter store 에 아래 인증정보가 먼저 입력되어 있어야 합니다.
- 01_preprocess_docs/01_parameter_store_example.ipynb 참고

In [20]:
import boto3
from utils.ssm import parameter_store

In [21]:
region=boto3.Session().region_name
pm = parameter_store(region)

In [22]:
opensearch_domain_endpoint = pm.get_params(
    key="opensearch_domain_endpoint",
    enc=False
)

opensearch_user_id = pm.get_params(
    key="opensearch_user_id",
    enc=False
)

opensearch_user_password = pm.get_params(
    key="opensearch_user_password",
    enc=True
)

In [23]:
opensearch_domain_endpoint = opensearch_domain_endpoint
rag_user_name = opensearch_user_id
rag_user_password = opensearch_user_password

http_auth = (rag_user_name, rag_user_password) # Master username, Master password

### Index 이름 셋팅
- 이전 노트북 01_preprocess_docs/02_load_docs_opensearch.ipynb를 통해서 생성된 OpenSearch Index name 입력

In [24]:
index_name = opensearch_user_password = pm.get_params(
    key="opensearch_index_name",
    enc=True
)

print (f'index_name: {index_name}')

index_name: v01-genai-poc-parent-doc-retriever


### OpenSearch Client 생성

In [25]:
from utils.opensearch import opensearch_utils

In [26]:
os_client = opensearch_utils.create_aws_opensearch_client(
    aws_region,
    opensearch_domain_endpoint,
    http_auth
)

## 6. Retriever based on Hybrid Search + RAG-Fusion + ReRanker
- LangChain에서 제공하는 **BaseRetriever** 클래스를 상속받아 **Custom Retriever**를 정의 할 수 있습니다.
- Hybrid-Search에 대한 자세한 내용는 **"01_rag_hybrid_search.ipyno"** 에서 확인 가능합니다.
- [RAG-Fusion](https://towardsdatascience.com/forget-rag-the-future-is-rag-fusion-1147298d8ad1)
![rag-fusion.png](../../../imgs/rag-fusion.png)

In [27]:
from utils.rag import OpenSearchHybridSearchRetriever

- 필터 설정 예시
- filter=[ <BR>
    　{"term": {"metadata.[**your_metadata_attribute_name**]": "**your first keyword**"}}, <BR>
    　{"term": {"metadata.[**your_metadata_attribute_name**]": "**your second keyword**"}},<BR>
]

In [28]:
opensearch_hybrid_retriever = OpenSearchHybridSearchRetriever(
    # necessary
    os_client=os_client,
    index_name=index_name,
    llm_text=llm_text, # llm for query augmentation in rag_fusion
    llm_emb=llm_emb,

    # option for lexical
    minimum_should_match=0,
    filter=[],

    # option for search
    fusion_algorithm="RRF", # ["RRF", "simple_weighted"], rank fusion 방식 정의
    ensemble_weights=[.5, .5], # [for lexical, for semantic], Lexical, Semantic search 결과에 대한 최종 반영 비율 정의
    reranker=True, # enable reranker with reranker model
    reranker_endpoint_name=endpoint_name, # endpoint name for reranking model
    rag_fusion=True, # enable rag_fusion
    query_augmentation_size=3, # query_augmentation_size in rag_fusion

    # option for async search
    async_mode=False,

    # option for output
    k=5, # 최종 Document 수 정의
    verbose=True
)

### Retrieval example
- default search

In [29]:
from utils.rag import show_context_used

In [30]:
query = "중지된 경우 이체"

In [31]:
%%time
search_hybrid_result = opensearch_hybrid_retriever.invoke(query)

print("\n==========  Results  ==========\n")
print(f'1. question: {query}')
print (f'2. # documents: {len(search_hybrid_result)}')
print("3. Documents: \n")

show_context_used(search_hybrid_result)



===== RAG-Fusion Queries =====
['중지된 경우 이체', '1. 자동이체 중단 시 대처 방법', '2. 계좌 이체 실패 원인과 해결책', '3. 송금 중지된 상황에서의 처리 절차']

[Exeeds ReRanker token limit] Number of chunk_docs after split and chunking= 2

##############################
async_mode
##############################
False
##############################
reranker
##############################
True
##############################
rag_fusion
##############################
True
##############################
HyDE
##############################
False
##############################
parent_document
##############################
False
##############################
complex_document
##############################
False
##############################
similar_docs_semantic
##############################

Score: 0.004098360655737705
['. Select the ACTIONS drop-down menu from the Devices page and select Manage device > Lock device. 3. Define the 350 character maximum Message devices receive informing them about their overdue payment and, if nec

- update parameters

In [32]:
opensearch_hybrid_retriever.update_search_params(
    k=10,
    minimum_should_match=30,
    filter=[
        #{"term": {"metadata.project": "KS"}},
    ],
    reranker=True,
    reranker_endpoint_name=endpoint_name,
    rag_fusion=True, # enable rag_fusion
    query_augmentation_size=2, # query_augmentation_size in rag_fusion
    llm_text=llm_text, # llm for query augmentation in rag_fusion
    verbose=False
)

In [33]:
query = "중지된 경우 이체"
search_hybrid_result = opensearch_hybrid_retriever.invoke(query)

print("\n==========  Results  ==========\n")
print(f'1. question: {query}')
print(f'2. # documents: {len(search_hybrid_result)}')
print("3. Documents: \n")

show_context_used(search_hybrid_result)


==========  Results  ==========

1. question: 중지된 경우 이체
2. # documents: 10
3. Documents: 


-----------------------------------------------
1. Chunk: 980 Characters
-----------------------------------------------
. A device can be unassigned from its assigned profile when the device is using a staggered license,
and the service period has ended, or when the device is using a non-staggered license, and the
30-day license expiration grace period has ended. Replace a license Complete the following steps if
you are: Switching all devices on one license to another (useful when replacing licenses that are
about to expire) 1. Select the license you would like to replace. 2. On the Licenses screen, select
Replace license. 3. Select the new license to replace the previous license. Switch configured
devices to a different license Complete the following steps if you are: Switching from a trial
license to a commercial license, Switching from a commercial license to a different commercial
license 

## 5. RAG using RetrievalQA powered by LangChain

In [35]:
from textwrap import dedent

### Prompting
- [TIP] Prompt의 instruction의 경우 한글보다 영어로 했을 때 더 좋은 결과를 얻을 수 있습니다.

In [36]:
system_prompt = dedent(
    """
    You are a master answer bot designed to answer user's questions.
    I'm going to give you contexts which consist of texts, tables and images.
    Read the contexts carefully, because I'm going to ask you a question about it.
    """
)

human_prompt = dedent(
    """
    Here is the contexts as texts: <contexts>{contexts}</contexts>

    First, find a few paragraphs or sentences from the contexts that are most relevant to answering the question.
    Then, answer the question as much as you can.

    Skip the preamble and go straight into the answer.
    Don't insert any XML tag such as <contexts> and </contexts> when answering.
    Answer in Korean.

    Here is the question: <question>{question}</question>

    If the question cannot be answered by the contexts, say "No relevant contexts".
    """
)

### Update Search Params (Optional)

In [37]:
from utils.rag import rag_chain
from langchain.schema.output_parser import StrOutputParser

In [38]:
opensearch_hybrid_retriever.update_search_params(
    k=5,
    minimum_should_match=0,
    filter=[],
    reranker=True,
    reranker_endpoint_name=endpoint_name,
    rag_fusion=True, # enable rag_fusion
    query_augmentation_size=2, # query_augmentation_size in rag_fusion
    llm_text=llm_text, # llm for query augmentation in rag_fusion
    
    # option for complex documents consisting of text, table and image
    complex_doc=False, # False 이면, invocation시 text만 넣는다.
    
    async_mode=True,
    verbose=False
)

### Request

In [41]:
qa = rag_chain(
    llm_text=llm_text,
    retriever=opensearch_hybrid_retriever,
    system_prompt=system_prompt,
    human_prompt=human_prompt,
    return_context=True,
    verbose=False,
    #multi_turn=True
)

In [42]:
#query = "중지된 경우 이체"
#query = "교육목표는?"
query = "vidio max size?"
#query = "수익률 그래프를 설명해줘"
response, contexts = qa.invoke(
    query = query
)
response

#show_context_used(contexts)

관련 정보를 찾았습니다. 해당 내용은 다음과 같습니다:

"You can add only one video file in the MP4 or MKV format. The video file must be less than 50 MB."

따라서 답변은 다음과 같습니다:

비디오 파일의 최대 크기는 50MB입니다. MP4나 MKV 형식의 비디오 파일 하나만 추가할 수 있으며, 이 파일은 50MB 미만이어야 합니다.

'관련 정보를 찾았습니다. 해당 내용은 다음과 같습니다:\n\n"You can add only one video file in the MP4 or MKV format. The video file must be less than 50 MB."\n\n따라서 답변은 다음과 같습니다:\n\n비디오 파일의 최대 크기는 50MB입니다. MP4나 MKV 형식의 비디오 파일 하나만 추가할 수 있으며, 이 파일은 50MB 미만이어야 합니다.'

In [43]:
print("##################################")
print("query: ", query)
print("##################################")

print (colored("\n\n### Answer ###", "blue"))
print_ww(response)

print (colored("\n\n### Contexts ###", "green"))
show_context_used(contexts)

##################################
query:  vidio max size?
##################################


### Answer ###
관련 정보를 찾았습니다. 해당 내용은 다음과 같습니다:

"You can add only one video file in the MP4 or MKV format. The video file must be less than 50 MB."

따라서 답변은 다음과 같습니다:

비디오 파일의 최대 크기는 50MB입니다. MP4나 MKV 형식의 비디오 파일 하나만 추가할 수 있으며, 이 파일은 50MB 미만이어야 합니다.


### Contexts ###

-----------------------------------------------
1. Chunk: 839 Characters
-----------------------------------------------
. You can add up to 10 image files in the PNG, JPG, JPEG, or GIF format (animated files are not
supported). Each image file must be less than 5 MB. To upload an image file, click Add and select a
file. To delete an image file, click next to the name of the uploaded image file. Note The device
control command must be transferred to the device to apply an image file to it. &gt;&gt;&gt; Video
Select a video file for the screen saver. You can add only one video file in the MP4 or MKV format.
The video file must be